In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import string
import tqdm
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from wordcloud import WordCloud
from wordcloud import STOPWORDS
from nltk.corpus import stopwords
import xgboost

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,HashingVectorizer
from sklearn import decomposition, ensemble
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score,f1_score

import dask
import multiprocessing
from joblib import delayed, Parallel, parallel_backend


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


### Loading data

In [2]:
cores = multiprocessing.cpu_count()

In [3]:
train_data = pd.read_csv("/content/I-Sarcasm_train_.csv")
test_data = pd.read_csv("/content/I-Sarcasm_test_.csv")

### Remove recurring tweets to prevent ambiguity

In [8]:
train_tweets=train_data['tweet'].tolist()
test_tweets=test_data['tweet'].tolist()

In [12]:
def keep_uniques(array, df):
    dels=[]
    for i in array:
        if array.count(i)>1:
            dels.append(i)
    dels=list(set(dels))
    for i in dels:
        df.drop( df[ df['tweet'] == i ].index, inplace=True)
    return df

In [13]:
train_data=keep_uniques(train_tweets, train_data)
test_data=keep_uniques(test_tweets, test_data)

In [19]:
train_data.drop("tweet_id",inplace=True,axis=1)
test_data.drop("tweet_id",inplace=True,axis=1)

In [20]:
len(train_data['tweet'].unique())

3473

In [21]:
len(test_data['tweet'].unique())

907

### Exploring Dataset

In [22]:
train_data.describe().T

,count,unique,top,freq
tweet,3473,3473,#RobertKraft and #TomBrady are innocent. They...,1
sarcasm_label,3473,2,not_sarcastic,2242
sarcasm_type,1230,7,sarcasm,854


In [23]:
train_data = train_data.sample(frac = 1)
test_data = test_data.sample(frac = 1)

In [24]:
train_data['sarcasm_label'].value_counts()

not_sarcastic    2242
sarcastic        1231
Name: sarcasm_label, dtype: int64

Here, we see that the `regular` class has 18k tweets, which causes our dataset to be imbalanced. So we shall delete some tweets from this class

In [25]:
# temp=train_data.loc[train_data['sarcasm_label'] == 'regular']
# lis=temp['tweets'].tolist()

# import random
# reg_del=[]
# visited=set()
# for _ in range(3600):
#     n=random.randint(0,18556)
#     if n not in visited:
#         reg_del.append(lis[n])
        
        
# for i in reg_del:
#     train_data.drop( train_data[ train_data['tweets'] == i ].index, inplace=True)

In [26]:
train_data['sarcasm_label'].value_counts()

not_sarcastic    2242
sarcastic        1231
Name: sarcasm_label, dtype: int64

In [27]:
test_data['sarcasm_label'].value_counts()

not_sarcastic    640
sarcastic        267
Name: sarcasm_label, dtype: int64

In [29]:
train_data.drop('sarcasm_type',axis=1,inplace=True)
test_data.drop("sarcasm_type",inplace=True,axis=1)

In [30]:
test_data = test_data.dropna()

## Data Cleaning & Preprocessing

In [31]:
def clean(tweet): 
    

    # Special characters
    tweet = re.sub(r"\x89Û_", "", tweet)
    tweet = re.sub(r"\x89ÛÒ", "", tweet)
    tweet = re.sub(r"\x89ÛÓ", "", tweet)
    tweet = re.sub(r"\x89ÛÏWhen", "When", tweet)
    tweet = re.sub(r"\x89ÛÏ", "", tweet)
    tweet = re.sub(r"China\x89Ûªs", "China's", tweet)
    tweet = re.sub(r"let\x89Ûªs", "let's", tweet)
    tweet = re.sub(r"\x89Û÷", "", tweet)
    tweet = re.sub(r"\x89Ûª", "", tweet)
    tweet = re.sub(r"\x89Û\x9d", "", tweet)
    tweet = re.sub(r"å_", "", tweet)
    tweet = re.sub(r"\x89Û¢", "", tweet)
    tweet = re.sub(r"\x89Û¢åÊ", "", tweet)
    tweet = re.sub(r"fromåÊwounds", "from wounds", tweet)
    tweet = re.sub(r"åÊ", "", tweet)
    tweet = re.sub(r"åÈ", "", tweet)
    tweet = re.sub(r"JapÌ_n", "Japan", tweet)    
    tweet = re.sub(r"Ì©", "e", tweet)
    tweet = re.sub(r"å¨", "", tweet)
    tweet = re.sub(r"SuruÌ¤", "Suruc", tweet)
    tweet = re.sub(r"åÇ", "", tweet)
    tweet = re.sub(r"å£3million", "3 million", tweet)
    tweet = re.sub(r"åÀ", "", tweet)
    
    #emojis
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    tweet =  emoji_pattern.sub(r'', tweet)
    
    # usernames mentions like "@abc123"
    ment = re.compile(r"(@[A-Za-z0-9]+)")
    tweet =  ment.sub(r'', tweet)
    
    # Contractions
    tweet = re.sub(r"he's", "he is", tweet)
    tweet = re.sub(r"there's", "there is", tweet)
    tweet = re.sub(r"We're", "We are", tweet)
    tweet = re.sub(r"That's", "That is", tweet)
    tweet = re.sub(r"won't", "will not", tweet)
    tweet = re.sub(r"they're", "they are", tweet)
    tweet = re.sub(r"Can't", "Cannot", tweet)
    tweet = re.sub(r"wasn't", "was not", tweet)
    tweet = re.sub(r"don\x89Ûªt", "do not", tweet)
    tweet = re.sub(r"aren't", "are not", tweet)
    tweet = re.sub(r"isn't", "is not", tweet)
    tweet = re.sub(r"What's", "What is", tweet)
    tweet = re.sub(r"haven't", "have not", tweet)
    tweet = re.sub(r"hasn't", "has not", tweet)
    tweet = re.sub(r"There's", "There is", tweet)
    tweet = re.sub(r"He's", "He is", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"You're", "You are", tweet)
    tweet = re.sub(r"I'M", "I am", tweet)
    tweet = re.sub(r"shouldn't", "should not", tweet)
    tweet = re.sub(r"wouldn't", "would not", tweet)
    tweet = re.sub(r"i'm", "I am", tweet)
    tweet = re.sub(r"I\x89Ûªm", "I am", tweet)
    tweet = re.sub(r"I'm", "I am", tweet)
    tweet = re.sub(r"Isn't", "is not", tweet)
    tweet = re.sub(r"Here's", "Here is", tweet)
    tweet = re.sub(r"you've", "you have", tweet)
    tweet = re.sub(r"you\x89Ûªve", "you have", tweet)
    tweet = re.sub(r"we're", "we are", tweet)
    tweet = re.sub(r"what's", "what is", tweet)
    tweet = re.sub(r"couldn't", "could not", tweet)
    tweet = re.sub(r"we've", "we have", tweet)
    tweet = re.sub(r"it\x89Ûªs", "it is", tweet)
    tweet = re.sub(r"doesn\x89Ûªt", "does not", tweet)
    tweet = re.sub(r"It\x89Ûªs", "It is", tweet)
    tweet = re.sub(r"Here\x89Ûªs", "Here is", tweet)
    tweet = re.sub(r"who's", "who is", tweet)
    tweet = re.sub(r"I\x89Ûªve", "I have", tweet)
    tweet = re.sub(r"y'all", "you all", tweet)
    tweet = re.sub(r"can\x89Ûªt", "cannot", tweet)
    tweet = re.sub(r"would've", "would have", tweet)
    tweet = re.sub(r"it'll", "it will", tweet)
    tweet = re.sub(r"we'll", "we will", tweet)
    tweet = re.sub(r"wouldn\x89Ûªt", "would not", tweet)
    tweet = re.sub(r"We've", "We have", tweet)
    tweet = re.sub(r"he'll", "he will", tweet)
    tweet = re.sub(r"Y'all", "You all", tweet)
    tweet = re.sub(r"Weren't", "Were not", tweet)
    tweet = re.sub(r"Didn't", "Did not", tweet)
    tweet = re.sub(r"they'll", "they will", tweet)
    tweet = re.sub(r"they'd", "they would", tweet)
    tweet = re.sub(r"DON'T", "DO NOT", tweet)
    tweet = re.sub(r"That\x89Ûªs", "That is", tweet)
    tweet = re.sub(r"they've", "they have", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"should've", "should have", tweet)
    tweet = re.sub(r"You\x89Ûªre", "You are", tweet)
    tweet = re.sub(r"where's", "where is", tweet)
    tweet = re.sub(r"Don\x89Ûªt", "Do not", tweet)
    tweet = re.sub(r"we'd", "we would", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"weren't", "were not", tweet)
    tweet = re.sub(r"They're", "They are", tweet)
    tweet = re.sub(r"Can\x89Ûªt", "Cannot", tweet)
    tweet = re.sub(r"you\x89Ûªll", "you will", tweet)
    tweet = re.sub(r"I\x89Ûªd", "I would", tweet)
    tweet = re.sub(r"let's", "let us", tweet)
    tweet = re.sub(r"it's", "it is", tweet)
    tweet = re.sub(r"can't", "cannot", tweet)
    tweet = re.sub(r"don't", "do not", tweet)
    tweet = re.sub(r"you're", "you are", tweet)
    tweet = re.sub(r"i've", "I have", tweet)
    tweet = re.sub(r"that's", "that is", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"doesn't", "does not", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"didn't", "did not", tweet)
    tweet = re.sub(r"ain't", "am not", tweet)
    tweet = re.sub(r"you'll", "you will", tweet)
    tweet = re.sub(r"I've", "I have", tweet)
    tweet = re.sub(r"Don't", "do not", tweet)
    tweet = re.sub(r"I'll", "I will", tweet)
    tweet = re.sub(r"I'd", "I would", tweet)
    tweet = re.sub(r"Let's", "Let us", tweet)
    tweet = re.sub(r"you'd", "You would", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"Ain't", "am not", tweet)
    tweet = re.sub(r"Haven't", "Have not", tweet)
    tweet = re.sub(r"Could've", "Could have", tweet)
    tweet = re.sub(r"youve", "you have", tweet)  
    tweet = re.sub(r"donå«t", "do not", tweet)   
            
    # Character entity references
    tweet = re.sub(r"&amp;", "&", tweet)
    
    # html tags
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    tweet = re.sub(html, '', tweet)
    
    # Urls
    tweet = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", tweet)
    tweet = re.sub(r'https?://\S+|www\.\S+','', tweet)
        
    #Punctuations and special characters
    
    tweet = re.sub('[%s]' % re.escape(string.punctuation),'',tweet)
    
    tweet = tweet.lower()
    
    splits = tweet.split()
    splits = [word for word in splits if word not in set(nltk.corpus.stopwords.words('english'))]
    tweet = ' '.join(splits)
    
    
    return tweet

In [33]:
train_data['cleaned_text']= train_data['tweet'].apply((lambda x: clean(x))) 
test_data['cleaned_text'] = test_data['tweet'].apply((lambda x: clean(x)))
print("Cleaned")

Cleaned


In [34]:
train_data.head()

,tweet,sarcasm_label,cleaned_text
2927,A funny thing happened to me. #Humor,not_sarcastic,funny thing happened humor
2588,I'm pretty sure this isn't right #a34 #Newbury...,not_sarcastic,pretty sure right a34 newbury accident late wo...
582,Can I pay anyone to put my bins out? This job ...,sarcastic,pay anyone put bins job hurts soul overstatement
2604,Daily #War on #Drugs is out! http://t.co/5Q9Ws...,not_sarcastic,daily war drugs stories via
2107,"@formetopoupon but Poupon, if they didn't have...",sarcastic,poupon boobies would know ladies literally way...


## EDA and Visualization

In [35]:
# sns.set(rc={'figure.figsize':(10,10)})
# sns.countplot(train_data['class'])

In [36]:
# from wordcloud import WordCloud
# stopwords = nltk.corpus.stopwords.words('english')

# plt.figure(figsize=(12,6))
# text = ' '.join(train_data.cleaned_text[train_data['class']=='regular'])
# wc = WordCloud(background_color='white',stopwords=stopwords).generate(text)
# plt.imshow(wc)
# plt.axis('off')
# plt.title('Regular Tweets',fontsize=25)

# plt.figure(figsize=(12,6))
# text = ' '.join(train_data.cleaned_text[train_data['class']=='irony'])
# wc1 = WordCloud(background_color='white',stopwords=stopwords).generate(text)
# plt.imshow(wc1)
# plt.axis('off')
# plt.title('Irony Tweets',fontsize=25)

# plt.figure(figsize=(12,6))
# text = ' '.join(train_data.cleaned_text[train_data['class']=='sarcasm'])
# wc2 = WordCloud(background_color='white',stopwords=stopwords).generate(text)
# plt.imshow(wc2)
# plt.axis('off')
# plt.title('Sarcasm Tweets',fontsize=25)

# plt.figure(figsize=(12,6))
# text = ' '.join(train_data.cleaned_text[train_data['class']=='figurative'])
# wc3 = WordCloud(background_color='white',stopwords=stopwords).generate(text)
# plt.imshow(wc3)
# plt.axis('off')
# plt.title('Figurative Tweets',fontsize=25)

### Encode our text classes

In [39]:
def encode_target(t_class):
    t_class=str(t_class)
    class_dict = {
        'not_sarcastic':0,
        'sarcastic':1
    }
    return class_dict[t_class]

In [40]:
train_data["target"] = train_data['sarcasm_label'].apply(lambda x: encode_target(x))
test_data["target"] = test_data['sarcasm_label'].apply(lambda x: encode_target(x))

### Preparing our train and test sets

In [41]:
train = train_data[['cleaned_text','target']]
train.columns = ['text','labels']

test = test_data[['cleaned_text','target']]
test.columns = ['text','labels']

In [42]:
train.head()

,text,labels
2927,funny thing happened humor,0
2588,pretty sure right a34 newbury accident late wo...,0
582,pay anyone put bins job hurts soul overstatement,1
2604,daily war drugs stories via,0
2107,poupon boobies would know ladies literally way...,1


In [43]:
test.head()

,text,labels
890,love parents get pissed dumb reason sarcasm,1
457,sneezing attack bad neck pain great day sarcasm,1
647,mexico love babies drugs,0
671,mood impressed ðÿ˜‚ðÿ˜‚ humor joke,0
116,michelle obama jokes life white house â€˜the l...,0


In [44]:
traintexts=train['text'].tolist()
testtexts=test['text'].tolist()

all_texts = traintexts + testtexts

## Feature Extraction

### 1. Count Vectors

In [45]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(all_texts)

# transform the training and test data using count vectorizer object
xtrain_count =  count_vect.transform(train['text'])
xtest_count =  count_vect.transform(test['text'])

### 2. Word level tf-idf

In [46]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=55000)
tfidf_vect.fit(all_texts)
xtrain_tfidf =  tfidf_vect.transform(train['text'])
xtest_tfidf =  tfidf_vect.transform(test['text'])

### 3. Hashing Vectorizer

In [47]:
hash_vectorizer = HashingVectorizer(n_features=55000)
hash_vectorizer.fit(all_texts)
xtrain_hash_vectorizer =  hash_vectorizer.transform(train['text']) 
xtest_hash_vectorizer =  hash_vectorizer.transform(test['text'])

## Building the Model

In [48]:
def train_model(classifier, feature_vector_train, label, feature_vector_test, test_y):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_test)
    
    return metrics.accuracy_score(predictions, test_y)

### Naive Bayes Classifier

In [49]:
print("NAIVE BAYES CLASSIFIER")
print("========================================================")
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train['labels'], xtest_count, test['labels'])
print("-> Count Vectors Accuracy: {}% ".format(round(accuracy,4)*100))

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train['labels'], xtest_tfidf, test['labels'])
print("-> WordLevel TF-IDF Accuracy: {}%".format(round(accuracy,4)*100))

# Naive Bayes on Hash Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_hash_vectorizer, train['labels'], xtest_hash_vectorizer, test['labels'])
# print("-> Hash Vectors Accuracy: ", round(accuracy,2))

NAIVE BAYES CLASSIFIER
-> Count Vectors Accuracy: 95.48% 
-> WordLevel TF-IDF Accuracy: 90.74%


### Linear Classifier

In [50]:
print("LINEAR CLASSIFIER")
print("========================================================")
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(solver="lbfgs",multi_class="auto",max_iter=13000, n_jobs=2), xtrain_count, train['labels'], xtest_count, test['labels'])
print("-> Count Vectors Accuracy: {}% ".format(round(accuracy,4)*100))

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(solver="lbfgs",multi_class="auto",max_iter=13000, n_jobs=2), xtrain_tfidf, train['labels'], xtest_tfidf, test['labels'])
print("-> WordLevel TF-IDF Accuracy: {}%".format(round(accuracy,4)*100))

# Linear Classifier on Hash Vectors
accuracy = train_model(linear_model.LogisticRegression(solver="lbfgs",multi_class="auto",max_iter=13000, n_jobs=2), xtrain_hash_vectorizer, train['labels'], xtest_hash_vectorizer, test['labels'])
print("-> Hash Vectors Accuracy: {}%".format(round(accuracy,4)*100))

LINEAR CLASSIFIER
-> Count Vectors Accuracy: 98.02% 
-> WordLevel TF-IDF Accuracy: 96.14%
-> Hash Vectors Accuracy: 97.46000000000001%


### XGBoost Classifier

In [51]:

print("XGBOOST CLASSIFIER")
print("========================================================")
# Linear Classifier on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count, train['labels'], xtest_count, test['labels'])
print("-> Count Vectors Accuracy: {}% ".format(round(accuracy,4)*100))

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf, train['labels'], xtest_tfidf, test['labels'])
print("-> WordLevel TF-IDF Accuracy: {}%".format(round(accuracy,4)*100))

# Linear Classifier on Hash Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_hash_vectorizer, train['labels'], xtest_hash_vectorizer, test['labels'])
print("-> Hash Vectors Accuracy: {}%".format(round(accuracy,4)*100))

XGBOOST CLASSIFIER
-> Count Vectors Accuracy: 98.24000000000001% 
-> WordLevel TF-IDF Accuracy: 98.24000000000001%
-> Hash Vectors Accuracy: 98.24000000000001%


In [52]:
from sklearn.neighbors import KNeighborsClassifier

In [53]:
print("KNN CLASSIFIER")
print("========================================================")
# Linear Classifier on Count Vectors
accuracy = train_model(KNeighborsClassifier(n_neighbors = 80, n_jobs=4) , xtrain_count, train['labels'], xtest_count, test['labels'])
print("-> Count Vectors Accuracy: {}% ".format(round(accuracy,4)*100))

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(KNeighborsClassifier(n_neighbors = 80, n_jobs=4) , xtrain_tfidf, train['labels'], xtest_tfidf, test['labels'])
print("-> WordLevel TF-IDF Accuracy: {}%".format(round(accuracy,4)*100))

# Linear Classifier on Hash Vectors
accuracy = train_model(KNeighborsClassifier(n_neighbors = 80, n_jobs=4) , xtrain_hash_vectorizer, train['labels'], xtest_hash_vectorizer, test['labels'])
print("-> Hash Vectors Accuracy: {}%".format(round(accuracy,4)*100))

KNN CLASSIFIER
-> Count Vectors Accuracy: 57.989999999999995% 
-> WordLevel TF-IDF Accuracy: 90.41%
-> Hash Vectors Accuracy: 96.25%


In [54]:
from sklearn.linear_model import SGDClassifier

In [55]:
print("SGD CLASSIFIER")
print("========================================================")
# Linear Classifier on Count Vectors
accuracy = train_model(SGDClassifier(max_iter=500, n_jobs=4) , xtrain_count, train['labels'], xtest_count, test['labels'])
print("-> Count Vectors Accuracy: {}% ".format(round(accuracy,4)*100))

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(SGDClassifier(max_iter=500, n_jobs=4) , xtrain_tfidf, train['labels'], xtest_tfidf, test['labels'])
print("-> WordLevel TF-IDF Accuracy: {}%".format(round(accuracy,4)*100))

# Linear Classifier on Hash Vectors
accuracy = train_model(SGDClassifier(max_iter=500, n_jobs=4) , xtrain_hash_vectorizer, train['labels'], xtest_hash_vectorizer, test['labels'])
print("-> Hash Vectors Accuracy: {}%".format(round(accuracy,4)*100))

SGD CLASSIFIER
-> Count Vectors Accuracy: 98.02% 
-> WordLevel TF-IDF Accuracy: 98.13%
-> Hash Vectors Accuracy: 98.24000000000001%


In [56]:
from sklearn.svm import LinearSVC

In [57]:
print("LINEAR SVC CLASSIFIER")
print("========================================================")
# Linear Classifier on Count Vectors
accuracy = train_model(LinearSVC(max_iter=3500) , xtrain_count, train['labels'], xtest_count, test['labels'])
print("-> Count Vectors Accuracy: {}% ".format(round(accuracy,4)*100))

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(LinearSVC(max_iter=3500) , xtrain_tfidf, train['labels'], xtest_tfidf, test['labels'])
print("-> WordLevel TF-IDF Accuracy: {}%".format(round(accuracy,4)*100))

# Linear Classifier on Hash Vectors
accuracy = train_model(LinearSVC(max_iter=3500) , xtrain_hash_vectorizer, train['labels'], xtest_hash_vectorizer, test['labels'])
print("-> Hash Vectors Accuracy: {}%".format(round(accuracy,4)*100))

LINEAR SVC CLASSIFIER
-> Count Vectors Accuracy: 98.13% 
-> WordLevel TF-IDF Accuracy: 98.13%
-> Hash Vectors Accuracy: 98.35000000000001%
